Externe bestanden connecten

In [14]:
import sqlite3
import pandas as pd
aenc_conn = sqlite3.connect("data/raw/aenc.sqlite")
adventureworks_conn = sqlite3.connect("data/raw/adventureworks.sqlite")  
northwind_conn = sqlite3.connect("data/raw/northwind1.sqlite") 




database connectie 

In [15]:
import pyodbc

DB = {'servername': r'Channie\SQLEXPRESS', 'database': 'sdm2' }

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' +
                              DB['servername'] + 
                              ';DATABASE=' + 
                              DB['database'] +
                              ';Trusted_Connection=yes'
)

export_cursor=export_conn.cursor()

Data ophalen, Pandas & SQL Queries.

In [16]:
import pandas as pd 

#bonus
df_bonus= pd.read_sql('select * from bonus',aenc_conn)
df_bonus



,emp_id,bonus_date,bonus_amount
0,129,31-dec-24,50
1,195,31-dec-23,50
2,299,31-dec-23,200
3,1142,31-dec-23,100
4,129,31-dec-22,50
5,129,31-dec-23,250
6,129,31-dec-24,400
7,195,31-dec-22,0
8,195,31-dec-23,450
9,195,31-dec-24,100


Database Reset

In [ ]:
import pyodbc

# Database connectie details
DB = {
    'servername': r'Channie\SQLEXPRESS',
    'database': 'sdm2'
}

# Maak een connectie met de database
export_conn = pyodbc.connect(
    'DRIVER={SQL Server};SERVER=' +
    DB['servername'] +
    ';DATABASE=' +
    DB['database'] +
    ';Trusted_Connection=yes'
)

export_cursor = export_conn.cursor()

# Tabellen in de juiste volgorde, zodat afhankelijkheden eerst worden verwijderd
tables = [
    "Sales_SalesTerritory",
    "sales_store",
    "Region",
    "Shippers",
    "Purchasing_Vendor",
    "Purchasing_PurchaseOrderHeader",
    "Purchasing_PurchaseOrderDetail",
    "Person_Person",
    "Person_Address",
    "CustomerDemographics",
    "state",
    "customer",
    "Sales_SalesOrderHeader",
    "Categories",
    "Sales_SalesOrderDetail",
    "Production_Product",
    "department",
    "employee",
    "bonus",
    "Production_BillOfMaterials"
]

# Verwijder de data uit elke tabel in de juiste volgorde
try:
    for table in tables:
        # Controleer of de tabel bestaat en verwijder data
        export_cursor.execute(f"IF OBJECT_ID('{table}', 'U') IS NOT NULL DELETE FROM {table};")
        print(f"Data verwijderd uit tabel: {table}")

    # Commit de wijzigingen
    export_conn.commit()
    print("Alle data is verwijderd. Klaar voor het importeren van nieuwe data.")

except Exception as e:
    # Als er een fout optreedt, rollback de transactie
    export_conn.rollback()
    print(f"Fout opgetreden: {e}")

finally:
    # Sluit de cursor en de connectie
    export_cursor.close()
    export_conn.close()


Data verwijderd uit tabel: bonus
Data verwijderd uit tabel: Sales_SalesOrderDetail
Data verwijderd uit tabel: Sales_SalesOrderHeader
Data verwijderd uit tabel: Purchasing_PurchaseOrderDetail
Data verwijderd uit tabel: Purchasing_PurchaseOrderHeader
Data verwijderd uit tabel: customer
Data verwijderd uit tabel: sales_store
Data verwijderd uit tabel: Production_Product
Data verwijderd uit tabel: Categories
Data verwijderd uit tabel: Person_Address
Data verwijderd uit tabel: Person_Person
Data verwijderd uit tabel: Purchasing_Vendor
Data verwijderd uit tabel: Shippers
Data verwijderd uit tabel: Sales_SalesTerritory
Data verwijderd uit tabel: Region
Data verwijderd uit tabel: state
Data verwijderd uit tabel: department
Data verwijderd uit tabel: employee
Alle data is verwijderd. Klaar voor het importeren van nieuwe data.


Bonus

In [17]:
import pyodbc

DB = {'servername': r'Channie\SQLEXPRESS', 'database': 'sdm2' }

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' +
                              DB['servername'] + 
                              ';DATABASE=' + 
                              DB['database'] +
                              ';Trusted_Connection=yes'
)

export_cursor=export_conn.cursor()
for index, row in df_bonus.iterrows():
    try:
        query = "INSERT INTO bonus (emp_id, bonus_date, bonus_amount) VALUES (?, ?, ?)"
        export_cursor.execute(query, (int(row['emp_id']), pd.to_datetime(row['bonus_date'], format="%d-%b-%y").strftime('%Y-%m-%d'), int(row['bonus_amount'])))
    except pyodbc.Error as e:
        print(f"Fout bij query: {query} met data {row.values}, fout: {e}")
        


export_conn.commit()
export_cursor.close()
export_conn.close()